- What were all the different types of fire calls in 2018?
- 
- What months within the year 2018 saw the highest number of fire calls?
- 
- Which neighborhood in San Francisco generated the most fire calls in 2018?
- 
- Which neighborhoods had the worst response times to fire calls in 2018?
- 
- Which week in the year in 2018 had the most fire calls?
- 
- Is there a correlation between neighborhood, zip code, and number of fire calls?
- 
- How can we use Parquet files or SQL tables to store this data and read it back?



In [2]:
ls data/ -alh

ls: -alh: No such file or directory
data/:
blogs.json         sf-fire-calls.csv


In [3]:
fire_calls_csv = "data/sf-fire-calls.csv"

In [55]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, corr, mean, weekofyear, year, to_date, to_timestamp, column

In [5]:
spark = SparkSession.builder.appName("FireCallsAnalysis").getOrCreate()

22/06/08 10:19:25 WARN Utils: Your hostname, Pauls-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.90 instead (on interface en0)
22/06/08 10:19:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/08 10:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/08 10:19:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [21]:
fire_df = spark.read.csv(fire_calls_csv, header=True)

In [23]:
fire_df.show(2)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+-----+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+------

In [24]:
fire_df.columns

['CallNumber',
 'UnitID',
 'IncidentNumber',
 'CallType',
 'CallDate',
 'WatchDate',
 'CallFinalDisposition',
 'AvailableDtTm',
 'Address',
 'City',
 'Zipcode',
 'Battalion',
 'StationArea',
 'Box',
 'OriginalPriority',
 'Priority',
 'FinalPriority',
 'ALSUnit',
 'CallTypeGroup',
 'NumAlarms',
 'UnitType',
 'UnitSequenceInCallDispatch',
 'FirePreventionDistrict',
 'SupervisorDistrict',
 'Neighborhood',
 'Location',
 'RowID',
 'Delay']

In [26]:
fire_df.groupby('CallTypeGroup').count().sort('count', ascending=False).show(truncate=False)

+----------------------------+-----+
|CallTypeGroup               |count|
+----------------------------+-----+
|null                        |99517|
|Potentially Life-Threatening|36734|
|Non Life-threatening        |18020|
|Alarm                       |17990|
|Fire                        |3035 |
+----------------------------+-----+



## What were all the different types of fire calls in 2018?

In [36]:
fire_df.groupby('CallType').count().sort('count', ascending=False).show(30, truncate=False)

+--------------------------------------------+------+
|CallType                                    |count |
+--------------------------------------------+------+
|Medical Incident                            |113794|
|Structure Fire                              |23319 |
|Alarms                                      |19406 |
|Traffic Collision                           |7013  |
|Citizen Assist / Service Call               |2524  |
|Other                                       |2166  |
|Outside Fire                                |2094  |
|Vehicle Fire                                |854   |
|Gas Leak (Natural and LP Gases)             |764   |
|Water Rescue                                |755   |
|Odor (Strange / Unknown)                    |490   |
|Electrical Hazard                           |482   |
|Elevator / Escalator Rescue                 |453   |
|Smoke Investigation (Outside)               |391   |
|Fuel Spill                                  |193   |
|HazMat                     

In [33]:
fire_df.select('CallType').distinct().sort('CallType').show(n=30, truncate=False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Administrative                              |
|Aircraft Emergency                          |
|Alarms                                      |
|Assist Police                               |
|Citizen Assist / Service Call               |
|Confined Space / Structure Collapse         |
|Electrical Hazard                           |
|Elevator / Escalator Rescue                 |
|Explosion                                   |
|Extrication / Entrapped (Machinery, Vehicle)|
|Fuel Spill                                  |
|Gas Leak (Natural and LP Gases)             |
|HazMat                                      |
|High Angle Rescue                           |
|Industrial Accidents                        |
|Marine Fire                                 |
|Medical Incident                            |
|Mutual Aid / Assist Outside Agency          |
|Odor (Strang

In [35]:
fire_df.select('CallType').distinct().count()

30

### What months within the year 2018 saw the highest number of fire calls?

In [37]:
fire_df.printSchema()

root
 |-- CallNumber: string (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: string (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: string (nullable = true)
 |-- ALSUnit: string (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: string (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: string (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- Sup

In [39]:
fire_df.show(1, vertical=True, truncate=False)

-RECORD 0-----------------------------------------------------------
 CallNumber                 | 20110016                              
 UnitID                     | T13                                   
 IncidentNumber             | 2003235                               
 CallType                   | Structure Fire                        
 CallDate                   | 01/11/2002                            
 WatchDate                  | 01/10/2002                            
 CallFinalDisposition       | Other                                 
 AvailableDtTm              | 01/11/2002 01:51:44 AM                
 Address                    | 2000 Block of CALIFORNIA ST           
 City                       | SF                                    
 Zipcode                    | 94109                                 
 Battalion                  | B04                                   
 StationArea                | 38                                    
 Box                        | 3362

In [51]:
fire_df.groupby('CallDate').count().show()

+----------+-----+
|  CallDate|count|
+----------+-----+
|05/02/2002|   20|
|06/11/2002|   19|
|06/26/2005|   11|
|07/24/2005|   27|
|10/31/2002|   21|
|04/04/2003|   17|
|06/18/2003|   20|
|08/01/2003|   29|
|08/06/2003|   22|
|02/08/2002|   25|
|06/06/2002|   18|
|07/10/2002|   24|
|03/19/2003|   19|
|05/18/2003|   21|
|08/23/2003|   28|
|07/09/2005|   23|
|07/29/2005|   20|
|12/16/2005|   32|
|01/29/2002|   39|
|04/09/2002|   25|
+----------+-----+
only showing top 20 rows



In [63]:
fire_df.groupby(year(to_date(column("CallDate"), "m/d/y"))).count().show()

+------------------------------+-----+
|year(to_date(CallDate, m/d/y))|count|
+------------------------------+-----+
|                          2003| 8499|
|                          2006| 8174|
|                          2005| 8282|
|                          2002| 8090|
|                          2009| 8789|
|                          2010| 9341|
|                          2008| 8869|
|                          2013|10020|
|                          2012| 9674|
|                          2011| 9735|
|                          2015|11458|
|                          2014|10775|
|                          2018|10136|
|                          2016|11609|
|                          2017|12135|
|                          2007| 8255|
|                          2004| 8283|
|                          2001| 7713|
|                          2000| 5459|
+------------------------------+-----+

